In [ ]:
###################################################################
#NASA Turbofan Engine Degradation reliability and failure analysis#
###################################################################
#
# Data cleaning on the NASA Turbofan engine degradation data
# The following questions will be investigated
# - Which engines show the fastest degradation?
# - Which sensors are strongest failure indicators?
# - How long do engines typically operate before failure?

In [ ]:
# Create tables - engine_readings, engine_metadata, failure_events
# Data clean and prep - missing values, rename columns, convert timestamps/cycles,  rolling avgs, delts
# EDA - trends over time, time to failure, correlation between failure and sensors
# Feature creation - RUL, degradation, failure frequency per engine type
# Run SQL queries - avg cycles to failure by engine type, rank engine by degradation spead, top 10 sensors with early failure,
#                   rolling avg using windows functions, failure rate by batch/custom (save sql files and show in notebook)
# Dashboard

Hello world!
